In [1]:
from langchain_chroma import Chroma
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [2]:
emb = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
sv_2024_pdf_path = "../data/Sustainability_report_2024_kr.pdf"
sv_2025_pdf_path = "../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"

# 문서 가져오기

In [4]:
# 문서 가져오기
loader_2024 = PyPDFLoader(sv_2024_pdf_path)
docs_2024 = loader_2024.load()
print("2024년 문서 로드 완료")

loader_2025 = PyPDFLoader(sv_2025_pdf_path)
docs_2025 = loader_2025.load()
print("2025년 문서 로드 완료")

2024년 문서 로드 완료
2025년 문서 로드 완료


In [5]:
print(len(docs_2024))
print(len(docs_2025))

83
87


# 텍스트 스플리트

In [6]:
# 텍스트 스플리트
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliiter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunck_2024 = spliiter.split_documents(docs_2024)
chunck_2025 = spliiter.split_documents(docs_2025)

In [7]:
len(chunck_2024)

207

In [8]:
len(chunck_2025)

237

# 벡터 스토어 저장

In [9]:
# 벡터 스토어 저장
db_path = "../vectorstore/chromadb_store"

vs_2024 = Chroma.from_documents(
    documents=chunck_2024,
    embedding=emb,
    persist_directory=db_path,
    collection_name="samsung_2024"
)

In [10]:
vs_2025 = Chroma.from_documents(
    documents=chunck_2025,
    embedding=emb,
    persist_directory=db_path,
    collection_name="samsung_2025"
)

In [11]:
r_2024 = vs_2024.as_retriever(search_kwargs = {"k":5})
r_2025 = vs_2025.as_retriever(search_kwargs = {"k":5})

In [12]:
r_2024.invoke("2024년 삼성 경영 관련 내용 알려줘")

[Document(id='fac7d5cb-ac8e-46b3-9f21-8334522dd535', metadata={'total_pages': 83, 'trapped': '/False', 'moddate': '2024-11-25T11:10:46+09:00', 'page': 29, 'source': '../data/Sustainability_report_2024_kr.pdf', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'page_label': '30', 'producer': 'Adobe PDF Library 15.0'}, page_content='삼성전자 지속가능경영보고서 2024 30Our Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\n사회적 책임을 다하며 미래로 함께 나아갑니다.\nPeople\n31    임직원\n39    공급망\n45    사회공헌\n48    개인정보보호/보안\n50    고객의 안전/품질'),
 Document(id='2f96f1bf-a6b7-492d-a440-4c873e60ed38', metadata={'source': '../data/Sustainability_report_2024_kr.pdf', 'moddate': '2024-11-25T11:10:46+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'producer': 'Adobe PDF Library 15.0', 'trapped': '/False', 'creationdate': '2024-11-25T11:10:32+09:00', 'total_pages': 83, 'page': 81, 'page_label': '82'}, page_content='삼성전자 지속가능경영보고서 2024\n82\nOur Company Appen

In [13]:
r_2025.invoke("2025년 삼성 경영 관련 내용 알려줘")

[Document(id='176c5eff-fe0a-4024-aedf-dc0e9919c419', metadata={'page_label': '5', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page': 4, 'producer': 'Adobe PDF Library 15.0', 'trapped': '/False', 'creationdate': '2025-07-10T16:11:16+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'total_pages': 87, 'moddate': '2025-09-04T16:51:11+09:00'}, page_content='삼성전자 지속가능경영보고서 2025\n05\nOur Company AppendixFacts & Figures PrinciplePlanet People\n회사소개\nAbout Us\n삼성전자주식회사(이하 삼성전자)는 인재와 기술을 기반으로 최고의 제품과 서비스를 창출함으로써 인류사회에 기여하는 글로벌 초일류 기업을 지향합니다. 이를 위해 삼성전자가 지켜나갈 약속인 5가지 경영원칙 을 세부원칙과 \n행동지침으로 구체화하고, 삼성전자 임직원이 지켜야 할 행동규범 으로 제정하여 모든 경영활동의 기준으로 삼고 있습니다. 앞으로도 삼성전자는 조직문화에 5가지 핵심가치 를 내재화하여 지속적으로 성장해갈 것입니다.\n사업부문 및 글로벌 네트워크 소개\n삼성전자는 제품 특성에 따라 DX(Device eXperience)와 DS(Device Solutions) 2개 부문으로 나뉘어 독립적으로 운영되고 있습니다. DX부문은 스마트폰, 네트워크 시스템, 컴퓨터, TV, 냉장고, 세탁기, 에어컨, 의료기기 등 완제품을 \n생산·판매하고 있으며, DS부문은 메모리 사업, Foundry 사업, System LSI 사업으로 구성되어 DRAM, NAND Flash, 모바일AP 등의 반도체 부품을 생산

In [14]:
esb_r = EnsembleRetriever(retrievers=[r_2024, r_2025], weights=[0.5, 0.5])
result = esb_r.invoke("2024년과 2025년 차이 요약해줘")
print(result[0].page_content)

삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People
CEO 메시지
Message from 
Our CEO
주주, 고객, 협력회사, 그리고 임직원 여러분,
2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 
거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 
심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 
어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3
조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 
리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 
아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 
깊이 감사 드립니다.
급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 
분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 
활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 
국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 
6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능성 보고지침
(CSRD)과 미국 증권거래위원회(SEC) 기후공시 기준 역시 세부 내용을 
순차적으로 확정하며 ESG 정보의 의무 공시 시대가 열리고 있습니다.
이와 함께, EU 탄소국경조정제도(CBAM)와 EU 배터리규제 등을 통한 
환경규제 역시 지속 강화되는 추세이고, 독일에서는 공급망의 인권과 
근로환경 관리를 의무화하는 공급망실사법이 2023년 발효되었으며, 
2024년 5월 EU 공급망 실사지침(CSDDD)이 확정되는 등 인권 분야에 
대한 관심 또한 지속 고조되고 있습니다. 
삼성전자는 이러한 추세에 맞춰 지속가능한 미래를 위한 노력을 계속해


In [15]:
esb_r = EnsembleRetriever(retrievers=[r_2024, r_2025], weights=[0.3, 0.7])
result = esb_r.invoke("2024년과 2025년 차이 요약해줘")
print(result[0].page_content)

삼성전자 지속가능경영보고서 2025
62
경제성과
핵심 재무 성과
1)
2022년 2023년 2024년
매출액 조 원  302.2  258.9  300.9 
영업이익 조 원  43.4  6.6  32.7 
당기순이익 조 원  55.7  15.5  34.5 
1) 연결 재무제표 기준
사업부문별 매출
1)
2022년 2023년 2024년
[사업부문별 매출(절대값)]
DX부문 조 원  182.5  170.0  174.9 
DS부문 조 원  98.5  66.6  111.1 
SDC 조 원  34.4  31.0  29.2
Harman
2)
조 원  13.2  14.4  14.3 
[사업부문별 매출(비율)]
DX부문 %  56  60  53 
DS부문 %  30  24  34 
SDC %  10  11  9 
Harman %  4  5  4 
1) 순매출액 기준
2) 2016년 인수
지역별 매출
1)
2022년 2023년 2024년
[지역별 매출(절대값)]
미주 조 원  119.0  92.1  118.8 
유럽 조 원  50.3  48.1  50.1 
한국 조 원  48.7  45.6  39.8 
아시아·아프리카
2) 조 원  84.3  73.1  92.1 
[지역별 매출(비율)]
미주 %  39  35  39 
유럽 %  17  19  17 
한국 %  16  18  13 
아시아∙아프리카 %  28  28  31 
1) 순매출액 기준
2) 2024년 보고서부터 아시아·아프리카에 중국도 합산
경제 가치 분배 2022년 2023년 2024년
[협력회사] 
구매 비용 조 원  219.8  212.8  226.1 
[지역사회]
사회공헌 비용 조 원  0.4  0.4  0.3 
[주주·투자자] 
배당금 조 원  9.8  9.8  9.8 
배당성향 %  18  68  29 
[채권자] 
이자 비용 조 원  0.8  0.9  0.9 
[임직원] 
인건비 조 원  37 .6  38.0  40.5 
[정부]
조세공과금 조 원  13.0  8.2  8.2


# 크로마 DB 병합

In [16]:
import chromadb
from chromadb.config import Settings

db_path = "../vectorstore/chromadb_store"

client = chromadb.PersistentClient(path=db_path)

src24 = client.get_collection("samsung_2024")
src25 = client.get_collection("samsung_2025")

In [17]:
# 새롭게 전체 데이터베이스 생성
vs_all = Chroma(
    collection_name="samsung_all",
    persist_directory=db_path,
    embedding_function=emb,
)

In [18]:
dst = client.get_collection("samsung_all")

In [19]:
# 2. samsung_2024 → samsung_all 복사
batch = 1000
offset = 0
while True:
    batch_data = src24.get(
        include=["embeddings", "metadatas", "documents"],
        limit=batch, offset=offset
    )
    ids = batch_data["ids"]
    if not ids:
        break

    # ID 충돌 방지: 접두사 붙이기
    new_ids = [f"{src24.name}::{i}" for i in ids]

    dst.upsert(
        ids=new_ids,
        documents=batch_data["documents"],
        embeddings=batch_data["embeddings"],   # 이미 계산된 벡터 재사용
        metadatas=batch_data["metadatas"],
    )
    offset += len(ids)
    print(f"[{src24.name}] moved {offset} docs so far...")

[samsung_2024] moved 207 docs so far...


In [20]:
# 3. samsung_2025 → samsung_all 복사
batch = 1000
offset = 0
while True:
    batch_data = src25.get(
        include=["embeddings", "metadatas", "documents"],
        limit=batch, offset=offset
    )
    ids = batch_data["ids"]
    if not ids:
        break

    new_ids = [f"{src25.name}::{i}" for i in ids]

    dst.upsert(
        ids=new_ids,
        documents=batch_data["documents"],
        embeddings=batch_data["embeddings"],
        metadatas=batch_data["metadatas"],
    )
    offset += len(ids)
    print(f"[{src25.name}] moved {offset} docs so far...")

[samsung_2025] moved 237 docs so far...


In [21]:
vs_all._collection.count()

444

In [27]:
result = vs_all.similarity_search("2024년과 2025년 변화 요약", k=5)

In [28]:
result[0].page_content

"삼성전자 지속가능경영보고서 2025\n04\nOur Company AppendixFacts & Figures PrinciplePlanet People\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2024년은 글로벌 지정학적 리스크와 AI 기술의 성장 등 급격한 산업의 \n변화를 겪은 한 해였습니다. 산업계 전반에 걸친 경쟁은 한층 더 \n치열해지고 있으며, 새로운 기술은 사업 전반에 큰 변화를 요구하고 \n있습니다. \n삼성전자는 이러한 환경에서도 전년 대비 16%의 매출 증가와 5배에 \n달하는 영업이익을 기록했습니다. 전략적 시설투자와 연구개발 강화 등 \n지속 성장의 기반을 다지는 데에도 집중했습니다.\n다양한 분야의 이해관계자 여러분께서 삼성전자의 이러한 노력을 \n인정하고 성원해 주신 결과, 삼성전자의 브랜드 가치는 인터브랜드 평가 \n기준 사상 첫 1,000억 달러를 돌파하며 5년 연속 글로벌 5위로 평가 \n받았습니다. 진심으로 감사드립니다.\n급변하는 경영 환경과 사회∙경제적 리스크에 대응하기 위한 확실한 \n해법은 기본에 충실한 경영과 지속가능한 성장 기반 마련에 있습니다. \n삼성전자는 이러한 믿음을 바탕으로 사업경쟁력과 기술혁신의 원동력을 \n제고하기 위해 사회 전반에서 다양한 활동을 지속하고 있습니다.\n환경 분야에 대해 먼저 말씀드립니다. 삼성전자는 2022년 9월 발표한 '新\n환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 \n혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.\nDX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로 2024\n년 말 기준 전체 에너지의 93.4%가 재생에너지로 전환되었고, 대표 \n제품 모델에는 고효율 에너지 기술을 적용해 2019년 대비 평균 31.5%\n의 소비전력을 절감했습니다. 제품에 사용되는 플라스틱 부품의 31%에 \n재활용 소재를 적용하는 성과를 이뤘습니다.\nDS(Device Solutions)부문은 2050년 탄소중립 달성을